### Vectorize Data Sets

In [1]:
from numba import jit, autojit # accelerate for loops
import numpy as np
import numpy.random as npr
import re # regular expressions
import glob
import pandas as pd

In [2]:
###--- Methods ---###

#### bootstrapping: resampling (virtual example)

nb = 100  # how many samples ?
ln = 3000 # how many words for each sample ? (3000 ~ length of each text)

def btp_ori(tot,nb,ln):
    if len(tot) == 0: return np.empty(0)
    else:
        idx = npr.randint(0, len(tot), (nb,ln)) # randomly pick indices from the 'at' array, 'nb' rows
        return np.array(tot[idx])

#### extract function words
# function word list: Bei Yu 2012: Function Words for Chinese Authorship Attribution
fw1 = ["的","是","不","了","在","有","這","為","地","也","得","就","那","以","著","之","可","于"
       "麼","而","然","没","於","還","只","無","又","如","但","其","此","與","把","全","被","卻"]

# difficult word list: 司法院裁判書類通俗化小組
fw2 = ["殆","似","難認無過失","難認有理由","尚難謂為於法無違","非無理由","非有理由","尚非無疑","尚非不可能",
       "可徵","徵諸","洵堪","迭於","係經","係指","洵屬","伊","渠","致","爰","遽認","矧","略以","按","第按","是","茲",
       "揆諸","皆無該條適用","即","無庸","縱",
       "上開","足資","可稽","堪以認定","業據","云云","等語","之","次查","復查","再查","系爭","固非無見","旋","惟查","均係"]
fwt = fw1+fw2

@jit
def ext_fw(txt,fw):
    # txt = input texts; fw = function word list
    j = np.empty(0)
    for i in fw:
        l = re.findall(i,txt)
        j = np.concatenate((j,np.array(l)))
    return j

In [3]:
#### generate csv files
global file_idx
file_idx = 0
def fw_csv(path,typ):
    list_of_files = glob.glob(path)

    for file_n in list_of_files:
        with open(file_n,'r') as myfile:         
            sl = myfile.read().replace('\n','')   # returns a string
            at = np.array(re.findall(r'\S+', sl)) # returns an array of the above string
            
            ## all texts as one
            # at = np.concatenate((at,sa))
        
        if   typ == 0: btp = btp_ori(at,nb,ln) # virtual examples: # = nb, length = ln
        elif typ == 1: btp = np.array(at,ndmin=2) # for test data
            
        for i in range(btp.shape[0]):
            bts = " ".join(btp[i])
            fwr = ext_fw(bts,fw1+fw2)

            # frequency table
            fwr_str = "\t".join(fwr).split("\t")
            fwr_tb  = pd.Series(fwr_str).value_counts()

            # store in dictionary
            dict1 = {}
            dict1.update(fwr_tb)
            dict1.update({"NAME": "%s" %f})
            dict1.update({"file_n": file_idx})
            dict1.update({"total wc": at.shape[0]}) # word count
            tp.append(dict1)
        global file_idx
        file_idx+=1
       
    return tp

<ipython-input-3-f54af6633fac>:33: SyntaxWarning: name 'file_idx' is used prior to global declaration
  global file_idx


In [4]:
#### Orderring columns
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

def reord_col(df):
    for a in fwt:
        if a not in [x.encode('utf-8') for x in df.columns]:
            df = df.append(pd.DataFrame(columns=[a]))

    col = ["NAME","file_n","total wc"]+fwt
    df = df[col]
    return df

In [6]:
#########################
###-- Training Data --###
#########################

#### read corpus
list_of_jud   = ["翁岳生","城仲模","林永謀","王和雄","余雪明","廖義男","曾有田","楊仁壽",
                 "彭鳳至","賴英照","謝在全","徐璧湖","林子儀","許宗力","許玉秀","林錫堯",
                 "池啟明","李震山","蔡清遊","賴浩敏","蘇永欽","黃茂榮","陳新民","陳春生","陳敏","葉百修"]

tp = [] # an empty list
file_idx = 0
for f in list_of_jud:
    # for each author ...
    path = './segmented_txt_2/%s/*.txt' %f 
    rs   = fw_csv(path,0)
    
df = pd.DataFrame(rs) 
df = reord_col(df)
# output csv
df.to_csv("./vectorized.csv", encoding="utf-8")

---
### Examine results:

In [7]:
import pandas as pd
test = pd.read_csv("./vectorized.csv", encoding="utf-8")
test

,Unnamed: 0,NAME,file_n,total wc,的,是,不,了,在,有,...,等語,之.1,次查,復查,再查,系爭,固非無見,旋,惟查,均係
0,0,翁岳生,0,2991,19,38,38,NaN,9,65,...,NaN,424,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN
1,1,翁岳生,0,2991,18,24,53,NaN,15,88,...,NaN,352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,翁岳生,0,2991,19,32,46,NaN,18,64,...,NaN,400,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN
3,3,翁岳生,0,2991,29,16,42,NaN,23,78,...,NaN,374,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN
4,4,翁岳生,0,2991,13,28,58,NaN,20,58,...,NaN,344,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
5,5,翁岳生,0,2991,12,32,54,NaN,16,73,...,NaN,334,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
6,6,翁岳生,0,2991,19,18,42,NaN,15,67,...,NaN,354,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN
7,7,翁岳生,0,2991,15,36,49,NaN,16,62,...,NaN,348,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN
8,8,翁岳生,0,2991,17,22,55,NaN,13,59,...,NaN,334,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
9,9,翁岳生,0,2991,24,26,48,NaN,13,77,...,NaN,362,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN


---
### Generate testing data from judicial Yuan (cf. parser.ipynb)

In [8]:
#####################
###-- Test Data --###
#####################

tp = [] # an empty list
path = './test_d/*.txt'
f    = "test data"
rs   = fw_csv(path,1)
    
df = pd.DataFrame(rs)
df = reord_col(df)

# output csv
df.to_csv("./test_ds.csv", encoding="utf-8")

In [9]:
import pandas as pd
test = pd.read_csv("./test_ds.csv", encoding="utf-8")
test

,Unnamed: 0,NAME,file_n,total wc,的,是,不,了,在,有,...,等語,之.1,次查,復查,再查,系爭,固非無見,旋,惟查,均係
0,0,test data,431,674,1,4,8,NaN,4,13,...,NaN,44,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
1,1,test data,432,559,2,6,9,NaN,3,8,...,NaN,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,test data,433,513,1,2,9,NaN,4,8,...,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,test data,434,388,2,4,16,NaN,2,7,...,NaN,38,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
4,4,test data,435,554,2,4,10,1,2,10,...,NaN,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,test data,436,544,2,NaN,4,NaN,3,11,...,NaN,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,test data,437,472,1,4,7,NaN,3,6,...,NaN,38,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
7,7,test data,438,759,1,4,13,NaN,3,15,...,NaN,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,test data,439,252,NaN,NaN,8,NaN,4,6,...,NaN,16,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
9,9,test data,440,913,2,2,15,NaN,5,16,...,1,78,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
